In [1]:
# Run a mock of fractal-demos/examples/01, with an additional illumination correction task.

"""
NOTES
1. The reference cycle is currently identified by its name (the last part of the
   OME-Zarr image path). We can then change this into using some zarr metadata.
2. We don't know yet how the to include two versions of the same reference-cycle
   image in the image list (where one has registration=True). This means that the
   the current version does not support (yet) running another task after
   registration.
   In order to have a consistent prototype, here we follow the naive approach of
   also including a dummy (ref_path, ref_path) pair in the parallelization list
   produced by the init-registration task.
3. This mock assumes that the "registration" task takes care of all the relevant
   steps, including: computing shifts, preparing new ROI tables, writing new
   image data.
   The mock task simulates the writing of image data, but in principle it should
   also include writing of new ROI tables. We haven't explored yet possible
   image-list-related glitches occurring when considering tables.
""";

In [2]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST;

executor = ThreadPoolExecutor()


def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    prefix = "[image_data_exist_on_disk]"
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"{prefix} {image.path} contains data")
        else:
            print(f"{prefix} {image.path} does *not* contain data")
            all_images_have_data = False
    return all_images_have_data

{'meta': {}, 'new_filters': {}, 'function_non_parallel': <cyfunction create_ome_zarr at 0x762618cd3c60>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x762618d118a0>}
{'meta': {}, 'new_filters': {}, 'function_non_parallel': <cyfunction new_ome_zarr at 0x762618d10ee0>, 'function_parallel': <cyfunction maximum_intensity_projection at 0x762618d13100>}
{'meta': {}, 'new_filters': {'illumination_correction': True}, 'function_non_parallel': None, 'function_parallel': <cyfunction illumination_correction at 0x762618cd20c0>}
{'meta': {}, 'new_filters': {}, 'function_non_parallel': None, 'function_parallel': <cyfunction cellpose_segmentation at 0x762618d11ff0>}
{'meta': {}, 'new_filters': {}, 'function_non_parallel': <cyfunction create_ome_zarr_multiplex at 0x762618d106c0>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x762618d118a0>}
{'meta': {}, 'new_filters': {'illumination_correction': True}, 'function_non_parallel': <cyfunction init_channel_parallelization at 0x762618cd2810>

In [3]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmp86431ksm/zarr_dir'


In [4]:
# Run create-ome-zarr-multiplex
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_multiplex_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        )
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Print current dataset information
debug(dataset)

# We have 6 images (two wells, three cycles)
assert len(dataset.images) == 6
assert dataset.image_paths == [
    f"{zarr_dir}/my_plate.zarr/A/01/0",
    f"{zarr_dir}/my_plate.zarr/A/01/1",
    f"{zarr_dir}/my_plate.zarr/A/01/2",
    f"{zarr_dir}/my_plate.zarr/A/02/0",
    f"{zarr_dir}/my_plate.zarr/A/02/1",
    f"{zarr_dir}/my_plate.zarr/A/02/2",
]

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

[create_ome_zarr_multiplex] START
[create_ome_zarr_multiplex] image_dir='/tmp/input_images'
[create_ome_zarr_multiplex] zarr_dir='/tmp/tmp86431ksm/zarr_dir'
[create_ome_zarr_multiplex] zarr_path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr'
[create_ome_zarr_multiplex] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/1'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_1.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/2'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_2.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tm

In [5]:
# Run registration part 1
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["registration_part_1_compound"],
            args_non_parallel=dict(ref_acquisition=0),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The number of images has not changed
assert len(dataset.images) == 6

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

# Images have not changed
for image in dataset.images:
    assert image.attributes.get("registration") is None

/tmp/ipykernel_38177/1302584944.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            None,
            None,
        ],
        zarr_dir='/tmp/tmp86431ksm/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/2',
                attributes={
             

In [6]:
# Find registration consensus
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["find_registration_consensus"],
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The number of images has not changed
assert len(dataset.images) == 6

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

# Images have not changed
for image in dataset.images:
    assert image.attributes.get("registration") is None

/tmp/ipykernel_38177/3129502953.py:13 <module>
    dataset: Dataset(
        id=None,
        history=[
            None,
            None,
            'find_registration_consensus',
        ],
        zarr_dir='/tmp/tmp86431ksm/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/2',

In [7]:
# Apply registration to image
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["apply_registration_to_image"],
            args_parallel=dict(overwrite_input=True),
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The number of images has not changed
assert len(dataset.images) == 6

# Image data do exist on disk
assert image_data_exist_on_disk(dataset.images)

# Images have been overwritten, and then they are now marked with registration=True
for image in dataset.images:
    assert image.attributes.get("registration") is True

[registration] START
[registration] path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/2'
[registration] overwrite_input=True
[registration] Reading information from /tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/2/registration_table_final
[registration] START
[registration] path='/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/02/2'
[registration] overwrite_input=True
[registration] Reading information from /tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/02/2/registration_table_final
[registration] out={'edited_images': [{'path': '/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/01/2'}]}
[registration] END
[registration] out={'edited_images': [{'path': '/tmp/tmp86431ksm/zarr_dir/my_plate.zarr/A/02/2'}]}
[registration] END
/tmp/ipykernel_38177/3628990298.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            None,
            None,
            'find_registration_consensus',
            'apply_registration_to_image',
        ],
        zarr_dir='/tmp/tmp86431ksm/zarr_dir',
     

In [8]:
executor.shutdown()